# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Описание данных**
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

## Шаг 1. Открытие файла с данными и изучение общей информации

In [1]:
# чтение файла и сохранение его в датафрейме solvency

import pandas as pd
solvency = pd.read_csv('/datasets/data.csv')


# изучение общей информации и вывод на экран первых 10 записей датафрейма

solvency.info()
solvency.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Всего в таблице 12 столбцов, из них 2 столбца с вещественным типом данных, 5 столбцов с целочисленным типом данных, остальные столбцы - строки.
Количество значений в столбцах различается. Это говорит о том, что в данных есть отсутствующие значения.

Каждая строка таблицы содержит данные заёмщиков: количество детей, трудовой стаж в днях, пол, возраст, уровень образования, семейное положение, доходы и цели, на которые заёмщик брал кредит. Количество значений в столбцах различается, значит, в данных есть отсутствующие значения. В столбце 'days_employed' с информацией об общем трудовом стаже заёмщика в днях присутствуют отрицательные значения. Также видно, что в некоторых строках данные о стаже в днях противоречат здравому смыслу (так, если посмотреть на строку с номером 4, то получается, что человек трудился 340266/365 = 932 года).

В столбце 'education' есть значения, отличающиеся друг от друга только регистром. Для корректного поиска дубликатов в датафрейме их нужно привести к нижнему регистру. Столбец 'purpose' содержит различные по написанию, но одинаковые по смыслу цели получения кредита. Для ответа на поставленные вопросы необходимо будет привести их к одному виду.

Для поиска возможных зависимостей между возвратом кредита в срок и каждого из параметров - наличие детей, семейное положение, уровень дохода и цели кредита - нам интересны прежде всего столбцы 'children', 'family_status', 'income_type', 'dept', 'total_income', 'purpose'.

**Вывод**

На этом этапе обнаружились следующие проблемы, которые нужно решать:
- пропуски в столбце с данными о доходе;
- неинформативное название столбца с возрастом клиента;
- столбец с данными о стаже, содержащий некорректные данные и имеющий пропуски;
- разный регистр одинаковых по написанию слов в столбце образования;
- разные по написанию, но одинаковые по смыслу цели кредита.

## Шаг 2. Предобработка данных

### Переименование столбцов

Переименуем столбец с возрастом заёмщика в годах на 'age_years'.

In [2]:
# переименование столбцов методом .rename()
solvency.rename(columns={'dob_years': 'age_years'}, inplace=True)

# проверка результатов - перечень названий столбцов
print(solvency.columns)

Index(['children', 'days_employed', 'age_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')


### Обработка пропусков

In [3]:
# суммарное количество пропусков, выявленных методом isnull() в таблице solvency
solvency.isnull().sum()

children               0
days_employed       2174
age_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Количество пустых значений в столбцах с общим стажем 'days_employed' и доходом 'total_income' совпадают. Проверим, пропущены  ли в строках с пустыми значениями в столбце 'days_employed' также и значения столбца 'total_income', или равенство количества - это простое совпадение.

In [4]:
# посчитаем количество строк в датафрейме, где в каждой строке одновременно пропущены значения в столбцах 'days_employed'
# и 'total_income'
len(solvency[(solvency['days_employed'].isna() == True) & (solvency['total_income'].isna() == True)])

2174

Этот вывод будет полезен для упрощения написания кода при дальнейшем анализе пропусков.
Это может говорить о том, что заёмщик не сотрудник, а владелец бизнеса, поэтому стаж и доход не указан. Проверим это предположение и посмотрим, какие источники дохода имеют заёмщики с пропущенными значениями стажа и дохода.

In [5]:
# посчитаем количество уникальных значений в столбце 'income_type' по строкам, где значения дохода и стажа пропущены
# применим метод .value_counts()

solvency[(solvency['days_employed'].isna() == True)]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Видно, что только 509 человек попадают под данное выше определение (это 2,3%). Остальные - это сотрудники или пенсионеры. Наша гипотеза оказалась неверной. По всей видимости, пропуски в столбцах 'day_employed' и 'total_income' носят или случайный характер, или кредит выдавался заёмщикам без информации о стаже и доходе.

Разберёмся с отрицательными значениями столбца 'day_employed'. Для этого напишем функцию, возвращающую модуль числа и применим её к вышеуказанному столбцу.

In [6]:
# напишем функцию, принимающую как аргумент число и возвращающую его модуль

def abs_func(number):
    if number >= 0:
        return number
    else:
        return -number

    
# применим функцию к столбцу 'day_employed'
solvency['days_employed'] = solvency['days_employed'].apply(abs_func)

# проверим, все ли отрицательные значения ушли
print('Количество отрицательных значений стажа в столбце days_employed:', solvency[solvency['days_employed'] < 0]['days_employed'].count())

Количество отрицательных значений стажа в столбце days_employed: 0


Теперь посчитаем, сколько несоответствующих реальности значений есть в столбце 'day_employed', и выведем результат в процентах, а также оценим, как такие значения влияют на среднее арифметическое столбца.

In [7]:
qty_unreal = solvency[solvency['days_employed'] > 18250]['days_employed'].count() # количество некорректных значений
total = 21525 # общее количество значений в датафрейме
print('Доля значений стажа, противоречащих здравому смыслу: {:.0%}'.format(qty_unreal / total))

mean_days_employed_total = solvency['days_employed'].mean() # среднее значение столбца
if mean_days_employed_total > 18250:
    print ('Эти данные вносят значимый вклад в средний стаж, согласно которому получается, что в среднем человек трудится в течение {:.0f} лет'.format(mean_days_employed_total / 365))
    print('Поэтому для заполнения пропусков будем использовать среднее значение стажа по столбцу без учёта некорректных значений:')
    mean_days_employed = solvency[solvency['days_employed'] <= 18250]['days_employed'].mean() # среднее значение столбца без учёта некорректных значений
    print('{:.0f} дней'.format(mean_days_employed))
else:
    print('Всё в порядке, несмотря на присутствие больших значений средний стаж вполне реален и составляет {:.0} лет'.format(mean_days_employed_total / 365))
    print('Поэтому будем использовать это значение для заполнения пропусков в столбце days_employed')



Доля значений стажа, противоречащих здравому смыслу: 16%
Эти данные вносят значимый вклад в средний стаж, согласно которому получается, что в среднем человек трудится в течение 183 лет
Поэтому для заполнения пропусков будем использовать среднее значение стажа по столбцу без учёта некорректных значений:
2352 дней


Заполним пропуски в столбце 'days_employed' методом fillna()

In [8]:
solvency['days_employed'] = solvency['days_employed'].fillna(mean_days_employed)

#проверим, остались ли пропущенные значения
print('Количество пропущенных значений столбца days_employed после заполнения пропусков:', solvency[(solvency['days_employed'].isna() == True)]['days_employed'].count())

Количество пропущенных значений столбца days_employed после заполнения пропусков: 0


Корректировать слишком большие значения в стаже не будем, так как этот столбец не понадобится для ответов на вопросы заказчика.


Так как для анализа нам понадобиться столбец 'children', проверим, какие значения он содержит:

In [9]:
solvency['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В этом столбце у нас тоже обнаружились отрицательные значения. Применим к столбцу написанную выше функцию abs_func(), чтобы откорректировать такие значения. Отдельно стоит сказать про количество в 20 детей - наверняка в данные здесь закралась ошибка, и к числу добавили лишний ноль. Но постановка вопроса такова, что нужно найти зависимость между возвратом кредита и наличием детей, а не их количеством. Поэтому данный артефакт оставим без корректировки.

In [10]:
solvency['children'] = solvency['children'].apply(abs_func)

print('Проверка результата применения функции. Столбец children содержит следующие значения:')
solvency['children'].value_counts()

Проверка результата применения функции. Столбец children содержит следующие значения:


0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

Теперь перейдём к заполнению пропущенных значений в столбце 'total_income' (доход). Доход — это количественная переменная, поэтому пропуски заполняют характерными значениями. Среднее значение здесь не подойдёт, так как оно некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства, а с доходами это часто происходит. Поэтому заполним пропуски медианой. Найдём медиану для каждого типа дохода, и заполним пропуски соответственно типу дохода.

In [11]:
# найдём медиану для каждого типа дохода и сохраним результат в датафрейме median_income
median_income = solvency.groupby('income_type')['total_income'].median()
print('Медианные доходы заёмщиков по типам занятости:')
print(median_income)
print()

# пропуски заполняются методом .fillna() соответствующим значением из датафрейма median_income с помощью цикла:
for i in median_income.index:
    solvency.loc[solvency['income_type'] == i,'total_income'] = solvency.loc[solvency['income_type'] == i,'total_income'].fillna(median_income[i])

# проверим, остались ли пропущенные значения
print('Количество пропущенных значений столбца total_income после заполнения пропусков:', solvency[(solvency['total_income'].isna() == True)]['total_income'].count())


Медианные доходы заёмщиков по типам занятости:
income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Количество пропущенных значений столбца total_income после заполнения пропусков: 0


**Вывод**

В пунктах 2.1 и 2.2 был переименован столбец возраста клиента, а также были проанализированы и заполнены пропуски в двух столбцах - стаж в днях и доход, и были откорректированы отрицательные значения в столбце со стажем и количеством детей.

### Замена типа данных

В предыдущем пункте мы увидели, что столбцы 'days_employed' и 'total_income' имеют тип данных float64 (вещественное число). Но данные в этих столбцах удобнее и логичнее читать и анализировать как целочисленные значения. Заменим тип данных у этих двух столбцов на integer методом astype().

In [12]:
solvency['days_employed'] = solvency['days_employed'].astype('int')
solvency['total_income'] = solvency['total_income'].astype('int')

print('Тип данных изменён, проверим полученный результат методом .info():')
print()
solvency.info()

Тип данных изменён, проверим полученный результат методом .info():

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
age_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Для удобства и наглядности данные в столбцах 'days_employed' и 'total_income' теперь имеют целочисленный тип integer.

### Обработка дубликатов

Перейдём к обработке дубликатов. В Шаге 1 было замечено, что некоторые значения в столбце 'education' различаются только регистром. Перед поиском дубликатов приведём все значения в столбце с образованием заёмщика к нижнему регистру методом str.lower().

In [13]:
solvency['education'] = solvency['education'].str.lower()

print('Все значения в столбце education приведены к нижнему регистру. Проверим результат, вызвав применив метод .value_counts()')
print()
solvency['education'].value_counts()

Все значения в столбце education приведены к нижнему регистру. Проверим результат, вызвав применив метод .value_counts()



среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Теперь посмотрим, есть ли дубликаты в наших данных. Для этого применим последовательно методы .duplicates() и .sum(), чтобы увидеть их количество.

In [14]:
print('Количество дубликатов в датафрейме:', solvency.duplicated().sum())

Количество дубликатов в датафрейме: 71


Дубликаты могли возникнуть в данных из-за некорректной выгрузки или же из-за того, что система распознаёт одно и то же слово, напечатанное в разных регистрах, как два различных значения. Удалим дубликаты методом .drop_duplicates(). Чтобы получить датафрейм, очищенный от дубликатов и с корректной индексацией, применим также метод .reset_index().

In [15]:
solvency = solvency.drop_duplicates().reset_index(drop=True)

print('Дубликаты удалены. Проверим результат, посчитаем количество дубликатов.')
print('Количество дубликатов:', solvency.duplicated().sum())


Дубликаты удалены. Проверим результат, посчитаем количество дубликатов.
Количество дубликатов: 0


Явные дубликаты удалены. Теперь разберёмся со столбцом 'purpose'. Он содержит различные по написанию, но одинаковые по смыслу значения, например: "сыграть свадьбу" и "на проведение свадьбы", или "покупка жилья" и "покупка жилья для семьи". Применим к столбцу 'purpose' метод .unique() и выведем все уникальные значения этого столбца на экран.

In [16]:
solvency['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Из получившегося большого списка можно выделить 5 общих целей, на которые люди берут кредиты: 
- недвижимость;
- ремонт;
- автомобиль;
- образование;
- свадьба.

Эти слова встречаются в столбце 'purpose' в разных формах. Чтобы правильно разделить их на категории, применим лемматизацию.

**Вывод**

Явные дубликаты удалены, но мы столкнулись с разной формой записи целей кредита. Поэтому обратимся к лемматизации, чтобы сделать наши данные более удобными для анализа.

### Лемматизация

Импортируем библиотеку pymystem3.

In [17]:
from pymystem3 import Mystem
m = Mystem()

Лемматизируем значения целей кредита в столбце 'purpose' методом .lemmatize() и запишем результат в новый столбец 'purpose_category'. Так как метод возвращает список слов, то склеим результат для удобства дальнейшей обработки методом .join().

In [18]:
# определим функцию, принимающую как аргумент строку и возвращающую леммы
def lemmatize_func(str):
    lemmas = ''.join(m.lemmatize(str))
    return lemmas

solvency['purpose_category'] = solvency['purpose'].apply(lemmatize_func)

print('Проверим результат, применив к столбцу метод .value_counts()')
solvency['purpose_category'].value_counts()

Проверим результат, применив к столбцу метод .value_counts()


автомобиль\n                                972
свадьба\n                                   791
на проведение свадьба\n                     768
сыграть свадьба\n                           765
операция с недвижимость\n                   675
покупка коммерческий недвижимость\n         661
операция с жилье\n                          652
покупка жилье для сдача\n                   651
операция с коммерческий недвижимость\n      650
покупка жилье\n                             646
жилье\n                                     646
покупка жилье для семья\n                   638
строительство собственный недвижимость\n    635
недвижимость\n                              633
операция со свой недвижимость\n             627
строительство жилой недвижимость\n          624
покупка недвижимость\n                      621
покупка свой жилье\n                        620
строительство недвижимость\n                619
ремонт жилье\n                              607
покупка жилой недвижимость\n            

Теперь наведём порядок в новом столбце 'purpose_category', перезаписая каждое значение на одно из пяти: недвижимость, ремонт, автомобиль, образование или свадьба. Для этого напишем функцию, принимающую как аргумент строку. Если одно из 6 слов (жилье, недвижимость, ремонт, автомобиль, образование, свадьба) содержится в строке, то функция возвращает соответствующюю категорию.

In [19]:
# функция проверяет наличие каждого из 6 слов и выводит категорию

def category_through_word(str):
    if 'свадьба' in str:
        return 'свадьба'
    if 'ремонт' in str:
        return 'ремонт'
    if 'недвижимость' in str:
        return 'недвижимость'
    if 'жилье' in str:
        return 'недвижимость'
    if 'автомобиль' in str:
        return 'автомобиль'
    if 'образование' in str:
        return 'образование'
    else:
        return str
    
# заменим значения в столбце 'purpose_category' с помощью функции
solvency['purpose_category'] = solvency['purpose_category'].apply(category_through_word)

# проверим результат вызовом метода .value_counts()
solvency['purpose_category'].value_counts()

недвижимость    10204
автомобиль       4306
образование      4013
свадьба          2324
ремонт            607
Name: purpose_category, dtype: int64

**Вывод**

С помощью лемматизации все цели кредита разбиты на 5 основных категорий. Это поможет нам при анализе данных и ответе на вопрос, как влияют цели кредита на возврат его в срок. Ясно видно, что самая популярная цель кредита связана с недвижимостью. Самая малочисленная категория - ремонт, и это логично. Люди в большинстве своём не готовы делать ремонт в кредит.

### Категоризация данных

В предыдущем пункте категоризация данных по целям кредита уже произведена. В этом шаге категоризируем данные по уровню дохода. Для этого вызовем метод .quantile(), чтобы определить границы для категоризации уровня дохода

In [20]:
print(solvency['total_income'].quantile([0.25, 0.50, 0.75]))

0.25    107623.00
0.50    142594.00
0.75    195820.25
Name: total_income, dtype: float64


Исходя из полученных результатов определим 4 категории по уровню дохода:
- низкий: до 107623;
- средний: от 107624 до 142594;
- выше среднего: от 142595 до 195820;
- высокий: от 195821.

Определим функцию с числовым аргументом доход клиента, которая возвращает категорию уровня дохода. Результат работы функции сохраним в новый отдельный столбец 'income_category'

In [21]:
def income_category_group(income):
    if income <= 107623:
        return 'низкий'
    elif income > 107624 and income <= 142594:
        return 'средний'
    elif income > 142595 and income <= 195820:
        return 'выше среднего'
    else:
        return 'высокий'

# применим функцию к столбцу 'total_income' и сохраним результат в столбец 'income_category'
solvency['income_category'] = solvency['total_income'].apply(income_category_group)

# проверим результат вызовом метода .value_counts()
solvency['income_category'].value_counts()

средний          5479
высокий          5365
низкий           5364
выше среднего    5246
Name: income_category, dtype: int64

**Вывод**

Теперь каждого заемщика можно отнести к какой-либо категории по уровню дохода и цели кредита. Датафрейм готов к анализу и ответу на вопросы заказчика.

## Шаг 3. Ответьте на вопросы

**- Есть ли зависимость между наличием детей и возвратом кредита в срок?**

Разделим заёмщиков на 2 категории по наличию детей (если значение в стоблце 'children' равно 0, то детей нет, если не равно 0, то дети есть). Сохраним каждую категорию в новый датафрейм. Затем посчитаем долю клиентов, имеющих задолженность по кредиту в каждой категории.

In [28]:
# заёмщики, у которых нет детей
clients_no_children = solvency[solvency['children'] == 0]
debt_no_children_part = clients_no_children[clients_no_children['debt'] != 0]['debt'].count() / clients_no_children['debt'].count()


# заёмщики, имеющие детей
clients_children = solvency[solvency['children'] != 0]
debt_children_part = clients_children[clients_children['debt'] != 0]['debt'].count() / clients_children['debt'].count()


# Составление итоговой таблицы:
children_final = pd.DataFrame(
    {
        'Наличие детей': [
            'Есть дети',
            'Нет детей'
        ],
        'Количество должников': [
            clients_children[clients_children['debt'] != 0]['debt'].count(), clients_no_children[clients_no_children['debt'] != 0]['debt'].count()
        ],
        'Общее количество': [clients_children['debt'].count(), clients_no_children['debt'].count()],
        'Доля должников': [debt_children_part, debt_no_children_part],
    }
)

print(children_final)
print()
print('Доля имеющих детей клиентов, которые имели задолженность по возврату кредита: {:.2%}'.format(debt_children_part))
print('Доля бездетных клиентов, которые имели задолженность по возврату кредита: {:.2%}'.format(debt_no_children_part))

  Наличие детей  Количество должников  Общее количество  Доля должников
0     Есть дети                   678              7363        0.092082
1     Нет детей                  1063             14091        0.075438

Доля имеющих детей клиентов, которые имели задолженность по возврату кредита: 9.21%
Доля бездетных клиентов, которые имели задолженность по возврату кредита: 7.54%


**Вывод**

Бездетные клиенты имеют меньше проблем с возвратом кредита, чем клиенты с детьми.



**- Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Ответ на этот вопрос будет нагляднее продемонстрировать в сводной таблице. Для этого воспользуемся методом .pivot_table() и создадим две сводных таблицы по столбцу 'family_status': в первой посчитаем количество заёмщиков, которые имели задолженности по кредитам, а во второй - общее количество заёмщиков. Затем методом merge() присоединим вторую таблицу к первой по столбцу 'family_status'. Чтобы получить итоговую таблицу, добавим столбец с долей клиентов, имеющих просрочки.

In [29]:
# создание сводных таблиц
family_status_debt = solvency.pivot_table(index='family_status', values='debt', aggfunc='sum')
family_status_total = solvency.pivot_table(index='family_status', values='debt', aggfunc='count')

# создание итоговой таблицы - присоединение таблицы family_status_total к таблице family_status_dept
family_status_final = family_status_debt.merge(family_status_total, on='family_status', how='left')

# переименуем столбцы итоговой таблицы для большей наглядности результата
new_names_pivot = ['debt', 'total']
family_status_final.set_axis(new_names_pivot, axis="columns", inplace = True)

# добавим к итоговой таблице столбец 'debt_part' с долей клиентов, имеющих просрочки
family_status_final['debt_part'] = family_status_final['debt'] / family_status_final['total']

# выведем итоговую таблицу на экран, отсортировав её по столбцу 'debt_part' по убыванию значений,
# чтобы самые проблемные клиенты были сверху
print(family_status_final.sort_values(by = 'debt_part', ascending = False))

                       debt  total  debt_part
family_status                                
Не женат / не замужем   274   2810   0.097509
гражданский брак        388   4151   0.093471
женат / замужем         931  12339   0.075452
в разводе                85   1195   0.071130
вдовец / вдова           63    959   0.065693


**Вывод**

Самые проблемные клиенты в плане возврата кредитов - это холостые или состоящие в гражданском браке. А самые благонадёжные заёмщики - это те, которые состоят в зарегистрированном браке сейчас, либо состояли в браке в прошлом.



**- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Для ответа на этот вопрос также воспользуемся методом создания сводных таблиц, только теперь по столбцу 'income_category'. В остальном алгоритм тот же. 

In [30]:
# создание сводных таблиц
income_category_debt = solvency.pivot_table(index='income_category', values='debt', aggfunc='sum')
income_category_total = solvency.pivot_table(index='income_category', values='debt', aggfunc='count')

# создание итоговой таблицы - присоединение таблицы income_category_total к таблице income_category_dept
income_category_final = income_category_debt.merge(income_category_total, on='income_category', how='left')

# переименуем столбцы итоговой таблицы для большей наглядности результата
new_names_pivot_income = ['debt', 'total']
income_category_final.set_axis(new_names_pivot_income, axis="columns", inplace = True)

# добавим к итоговой таблице столбец 'debt_part' с долей клиентов, имеющих просрочки
income_category_final['debt_part'] = income_category_final['debt'] / income_category_final['total']

# выведем итоговую таблицу на экран, отсортировав её по столбцу 'debt_part' по убыванию значений,
# чтобы самые проблемные клиенты были сверху
print(income_category_final.sort_values(by = 'debt_part', ascending = False))

                 debt  total  debt_part
income_category                        
средний           483   5479   0.088155
выше среднего     448   5246   0.085398
низкий            427   5364   0.079605
высокий           383   5365   0.071389


**Вывод**

Люди со средним и выше среднего доходом допускют просрочки по погашению задолженностей чаще других. В то время как люди с низким доходом в плане возврата кредитов имеют лучшие показатели, чем "середнячки". Меньше всего невозвратов наблюдаем у людей высокого достатка.

**- Как разные цели кредита влияют на его возврат в срок?**

Воспользуемся уже отработанным алгоритмом из предыдущих двух пунктов. Формирование сводных таблиц ведётся по столбцу 'purpose_category'.

In [31]:
# создание сводных таблиц
purpose_category_debt = solvency.pivot_table(index='purpose_category', values='debt', aggfunc='sum')
purpose_category_total = solvency.pivot_table(index='purpose_category', values='debt', aggfunc='count')

# создание итоговой таблицы - присоединение таблицы purpose_category_total к таблице purpose_category_dept
purpose_category_final = purpose_category_debt.merge(purpose_category_total, on='purpose_category', how='left')

# переименуем столбцы итоговой таблицы для большей наглядности результата
new_names_pivot_purpose = ['debt', 'total']
purpose_category_final.set_axis(new_names_pivot_purpose, axis="columns", inplace = True)

# добавим к итоговой таблице столбец 'debt_part' с долей клиентов, имеющих просрочки
purpose_category_final['debt_part'] = purpose_category_final['debt'] / purpose_category_final['total']

# выведем итоговую таблицу на экран, отсортировав её по столбцу 'debt_part' по убыванию значений,
# чтобы самые проблемные клиенты были сверху
print(purpose_category_final.sort_values(by = 'debt_part', ascending = False))

                  debt  total  debt_part
purpose_category                        
автомобиль         403   4306   0.093590
образование        370   4013   0.092200
свадьба            186   2324   0.080034
недвижимость       747  10204   0.073207
ремонт              35    607   0.057661


**Вывод**

В топе списка - "автомобиль" и "образование". По этим целям в 9,4% и 9,2% случаев соответственно случаются просрочки платежей. Из оформляющих кредит на свадьбу 8% имеют проблемы с задолженностями. Немного ниже эта цифра в ситуации с недвижимостью - 7,3%. А вот те, кто планирует на кредитные деньги сделать ремонт - самые благонадёжные, среди них лишь 5,8% задерживают оплату.

## Шаг 4. Общий вывод

Результаты получились очень интересные. С одной стороны, заёмщики, состоящие сейчас или состоявшие ранее в браке в целом более благонадёжные, чем одинокие или проживающие в гражданском браке. В то же время люди с детьми чаще допускают просрочки по возврату кредита, чем бездетные.

Что касается уровня дохода, зависимость получилась нелинейная (хотя можно было бы предположить обратное) - больше всего задолженностей случается у людей со средним и выше среднего достатком, а заёмщики с низким доходом имеют меньше проблем с возвратом кредита в срок, а у заёмщиков с высоким доходом - самые лучшие из всех категорий показатели.

Если смотреть в разрезе целей кредита, то меньше всего подводят заёмщики, оформляющие ссуду на ремонт. А вот автолюбители и те, кто берёт кредит на образование чаще других погашают позже срока. По середине - цели, связанные со свадьбой и недвижимостью, на их долю приходится 8% и 7,3% невозвратов соответственно.